In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats
from bs4 import BeautifulSoup as bs

#secret = 'sk_34b72a0f91dc481493aa037abf58e046'
secret = 'sk_d9fd3e6d4d4343deb69e1ddad8f1ade2'

In [2]:
#scrape webpage

def get_spy():
    
    url = 'https://www.slickcharts.com/sp500'

    request = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})

    soup = bs(request.text, "lxml")

    stats = soup.find('table',class_='table table-hover table-borderless table-sm')

    df =pd.read_html(str(stats))[0]

    df['% Chg'] = df['% Chg'].str.strip('()-%')

    df['% Chg'] = pd.to_numeric(df['% Chg'])

    df['Chg'] = pd.to_numeric(df['Chg'])

    return df


In [3]:
df1 = pd.DataFrame(get_spy()['Symbol'])
df1.rename(columns = {'Symbol':'Ticker'}, inplace = True)
df1

,Ticker
0,AAPL
1,MSFT
2,AMZN
3,GOOGL
4,GOOG
...,...
499,PVH
500,FOX
501,DISH
502,NWS


In [4]:
symbol = 'AAPL' 
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_34b72a0f91dc481493aa037abf58e046' 
data = requests.get(api_url).json()

In [5]:
data

{'companyName': 'Apple Inc',
 'marketcap': 2292792740460,
 'week52high': 182.44,
 'week52low': 129.04,
 'week52highSplitAdjustOnly': 182.94,
 'week52lowSplitAdjustOnly': 129.04,
 'week52change': 0.0703130548575106,
 'sharesOutstanding': 16185181000,
 'float': 0,
 'avg10Volume': 90096123,
 'avg30Volume': 86127015,
 'day200MovingAvg': 162.08,
 'day50MovingAvg': 142.43,
 'employees': 147000,
 'ttmEPS': 6.15,
 'ttmDividendRate': 0.8881528413776449,
 'dividendYield': 0.006269609214864076,
 'nextDividendDate': '',
 'exDividendDate': '2022-05-06',
 'nextEarningsDate': '2022-07-26',
 'peRatio': 22.49269377995781,
 'beta': 1.2682786675878701,
 'maxChangePercent': 53.68020226193693,
 'year5ChangePercent': 3.091901697304417,
 'year2ChangePercent': 0.5725165926809042,
 'year1ChangePercent': 0.0703130548575106,
 'ytdChangePercent': -0.2000402069301613,
 'month6ChangePercent': -0.19418603460574516,
 'month3ChangePercent': -0.1880259331883939,
 'month1ChangePercent': 0.008112724167378182,
 'day30Chan

In [6]:
data['year1ChangePercent']

0.0703130548575106

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:
symbol_groups = list(chunks(df1['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Two-Year Price Return', 'Number of Shares to Buy']

final_dataframe = pd.DataFrame(columns = my_columns)

In [9]:
final_dataframe

,Ticker,Price,One-Year Price Return,Two-Year Price Return,Number of Shares to Buy


In [11]:

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={secret}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   data[symbol]['stats']['year2ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe     

C:\Users\zvern\AppData\Local\Temp\ipykernel_18396\2253992903.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_18396\2253992903.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_18396\2253992903.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_18396\2253992903.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\L

,Ticker,Price,One-Year Price Return,Two-Year Price Return,Number of Shares to Buy
0,AAPL,141.66,0.070313,0.572517,N/A
1,MSFT,267.7,0.010112,0.336228,N/A
2,AMZN,116.67,-0.315235,-0.154426,N/A
3,GOOGL,2359.5,-0.037006,0.637291,N/A
4,GOOG,2370.76,-0.066593,0.644842,N/A
...,...,...,...,...,...
499,PVH,63.35,-0.42805,0.353135,N/A
500,FOX,30.35,-0.154806,0.166074,N/A
501,DISH,18.85,-0.555529,-0.44346,N/A
502,NWS,16.08,-0.329021,0.393777,N/A


In [ ]:
final_dataframe

In [12]:
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe.head(5)

,Ticker,Price,One-Year Price Return,Two-Year Price Return,Number of Shares to Buy
0,AAPL,141.66,0.070313,0.572517,N/A
1,MSFT,267.7,0.010112,0.336228,N/A
2,AMZN,116.67,-0.315235,-0.154426,N/A
3,GOOGL,2359.5,-0.037006,0.637291,N/A
4,GOOG,2370.76,-0.066593,0.644842,N/A


In [13]:
def portfolio_input():
  global portfolio_size
  portfolio_size = input('Enter the value of your portfolio')
  global val

  try:
    val = float(portfolio_size)
    print(portfolio_size)
  
  except ValueError:
    print('Please enter a number \n Try again')

In [14]:
portfolio_input()

Enter the value of your portfolio 10000


10000


In [15]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
    


In [16]:
writer = pd.ExcelWriter('Momentum_Trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Momentum Trades', index = False)

In [17]:

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:

column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['One-Year Price Return', string_format],
                    'D':['Two-Year Price Return', string_format],
                    'E': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [19]:

writer.save()